In [1]:
import meep as mp
from meep import mpb
import numpy as np
import scipy.ndimage
import math
import matplotlib.pyplot as plt
from itertools import product
import time
import timeit

In [2]:
def get_material_region(eps):
    material_region = 0.0 * eps
    resolution = eps.shape[0]
    for i in range(resolution):
        for j in range(resolution):
            for k in range(resolution):
                if eps[i,j,k] != 1:
                    material_region[i,j,k] = 1
    return material_region

def get_material_region_1D(eps):
    material_region = 0.0 * eps
    resolution = eps.shape[0]
    for i in range(resolution):
        if eps[i] != 1:
            material_region[i] = 1
    return material_region

In [3]:
def get_mode_info(ms, k_point, pol="none"):
    k_points = [k_point]
    efields = []

    #for e field acquisition
    def get_efields(ms, band):
        efields.append(ms.get_efield(band, bloch_phase=True))
    ms.k_points = [k_point]
    if pol == "none":
        ms.run(mpb.output_at_kpoint(k_point), mpb.fix_efield_phase,
                  get_efields)
    elif pol == "te":
        ms.run_te(mpb.output_at_kpoint(k_point), mpb.fix_efield_phase,
                  get_efields)
    elif pol == "tm":
        ms.run_tm(mpb.output_at_kpoint(k_point), mpb.fix_efield_phase,
                  get_efields)

    converted_Ex = []
    converted_Ey = []
    converted_Ez = [] 
    for f in efields:
        # Get just the x component of the efields
        Ex = f[..., 0]
        Ey = f[..., 1]
        Ez = f[..., 2]
        converted_Ex.append(Ex)
        converted_Ey.append(Ey)
        converted_Ez.append(Ez)

    E = np.array([converted_Ex,converted_Ey, converted_Ez])
    E = np.swapaxes(E, 0, 1)
    omega = ms.all_freqs[0]
    vg = ms.compute_group_velocities()
    vgmags = np.array([np.linalg.norm(v) for v in vg])
    return E, omega, vgmags




# N is number of points to keep
# vol is volume of the unit cell, in cartesian (include the jacobian)
def get_M_grid(Ez, material_region, N, vol):
    res = material_region.shape[0]
    ft = np.abs(np.fft.fftshift(np.fft.fftn(np.multiply(Ez, material_region))))
    # 0 is now at index res/2 = 8
    # res must be even for this to work, and N odd
    lind = res//2 - (N-1)//2 # = 8 - 3 = 5
    uind = res//2 + (N-1)//2 + 1 # 8 + 3 + 1 = 12
    ft_reduce = ft[lind:uind, lind:uind, lind:uind]/(res)**3*vol
    return ft_reduce
    #return ft/(res)**3 #(0,0,0) is stored at (3,3,3) for resolution of 16

def get_M_grid_1D(Ez, material_region, N, vol):
    res = material_region.shape[0]
    ft = np.abs(np.fft.fftshift(np.fft.fftn(np.multiply(Ez, material_region))))
    # 0 is now at index res/2 = 8
    # res must be even for this to work, and N odd
    lind = res//2 - (N-1)//2 # = 8 - 3 = 5
    uind = res//2 + (N-1)//2 + 1 # 8 + 3 + 1 = 12
    ft_reduce = ft[lind:uind]/(res)*vol
    return ft_reduce
    

    

BCC_basis = 1/2 * np.array([[1,1,-1], [-1,1,1], [1,-1,1]])
sign_options = list(product([-1,0,1],[-1,0,1],[-1,0,1]))
BCC_r_basis = np.array([[1,1,0],[0,1,1],[1,0,1]])
lattice_to_cartesian_BCC = BCC_basis
BCC_to_cartesian_r = np.array([[1,0,1],[1,1,0],[0,1,1]])
BCC_to_cart_r = np.array([[1,0,1],[1,1,0],[0,1,1]])
cart_to_BCC_r = 0.5 * np.array([[1,1,-1],[-1,1,1],[1,-1,1]])

def in_ws_cell_BCC(r):
    r = np.array(r)
    for s in sign_options:
        if s[0] == s[1] and s[1] == s[2] and s[2] == 0:
            continue
        a = np.sum(np.multiply(BCC_basis.T, s).T, axis=0)
        if ((r-0.5*a) @ (a/np.linalg.norm(a)).T) > 0:
            return False
    return True

def in_1BZ_BCC(k):
    k = np.array(k)
    for s in sign_options:
        if s[0] == s[1] and s[1] == s[2] and s[2] == 0:
            continue
        a = np.sum(np.multiply(BCC_r_basis.T, s).T, axis=0)
        if ((k-0.5*a) @ (a/np.linalg.norm(a)).T) > 0:
            return False
    return True

def near_1BZ_BCC(k):
    k = np.array(k)
    for s in sign_options:
        if in_1BZ_BCC(0.15*np.array(s) + k):
            return True
    return False

FCC_to_cartesian_r = np.array([[1, 1, -1],[-1, 1, 1],[1, -1, 1]])
cartesian_to_FCC_r = 1/2 * np.array([[1, 0, 1,], [1, 1, 0], [0, 1, 1]])
FCC_r_basis = np.array([[1, -1, 1], [1, 1, -1], [-1, 1, 1]])
FCC_basis = np.array([[1/2, 0, 1/2], [1/2, 1/2, 0], [0, 1/2, 1/2]])

def in_1BZ_FCC(k):
    k = np.array(k)
    for s in sign_options:
        if s[0] == s[1] and s[1] == s[2] and s[2] == 0:
            continue
        a = np.sum(np.multiply(FCC_r_basis.T, s).T, axis=0)
        if ((k-0.5*a) @ (a/np.linalg.norm(a)).T) > 0:
            return False
    return True

def near_1BZ_FCC(k):
    k = np.array(k)
    for s in sign_options:
        if in_1BZ_FCC(0.15*np.array(s) + k):
            return True
    return False


## SC

In [ ]:
#messing around with 3D now
geometry_lattice = mp.Lattice(size=mp.Vector3(1,1,1))
resolution = 16

Gamma = mp.Vector3(0,0,0)
X = mp.Vector3(0.5,0,0)
M = mp.Vector3(0.5,0.5,0)
K = mp.Vector3(0.3,0.4,0.5)

# k_points = [M, X, Gamma, K, M, Gamma]
k_points = [M, Gamma, X, M]
k_points = mp.interpolate(8, k_points)

#add a material cube to center
geometry = [mp.Block(center=mp.Vector3(0.1, 0.1, 0.1), material=mp.Medium(epsilon=1.0), size=mp.Vector3(0.1, 0.1, 0.1))]
# geometry = [mp.Block(center=mp.Vector3(0.5, 0.5, 0.5), material=mp.Medium(epsilon=12.0), size=mp.Vector3(1,1,1))]
#geometry = [mp.Block(center=mp.Vector3(0,0,0), material=mp.Medium(epsilon=10.0), size=mp.Vector3(0.2, 0.2, 0.2))]
rsphere = 0.2
# geometry = [mp.Sphere(rsphere, center=mp.Vector3(0,0,0), material=mp.Medium(epsilon=12.0))]
num_bands = 12
                      
ms = mpb.ModeSolver(
    geometry=geometry,
    default_material=mp.Medium(epsilon=12.0),
    geometry_lattice=geometry_lattice,
    k_points=k_points,
    resolution=resolution,
    num_bands=num_bands,
    dimensions=3
)

ms.run()
eps = np.array(ms.get_epsilon(), dtype=complex)
material_region = get_material_region(eps)
all_freqs = ms.all_freqs


ms.run_tm()
tm_freqs = ms.all_freqs

# ms.run()
# all_freqs = ms.all_freqs

# ms.run_tm()
# eps = np.array(ms.get_epsilon(), dtype=complex)
# material_region = get_material_region(eps)
# imaterial_region = inv_material_region(eps)

In [ ]:
np.savez("phc_an_files/phc3d_sc_fs_bands_e_12_12.npz", ms.all_freqs)

In [ ]:
num_sample = 15
BZ_sample = np.linspace(0,0.5, num_sample)
print(BZ_sample)

# number of points in mode integral to keep (which is in Fourier space, remember)
N_offset = 7

Mx_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
My_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
Mz_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
omega_vals = np.zeros([num_bands, num_sample, num_sample, num_sample])
vg_vals = np.zeros([num_bands, num_sample, num_sample, num_sample])


for (l, kx) in enumerate(BZ_sample):
    for (m, ky) in enumerate(BZ_sample):
        for (n, kz) in enumerate(BZ_sample):
            k = mp.Vector3(kx, ky, kz)
            E, omega, vgmag = get_mode_info(ms, k)
            Mbands = []
            for i in range(num_bands):
                Ex = np.array(E[i,0,:,:,:])
                Ey = np.array(E[i,1,:,:,:])
                Ez = np.array(E[i,2,:,:,:])
                Mx = get_M_grid(Ex, material_region, N_offset,1)
                My = get_M_grid(Ey, material_region, N_offset,1)
                Mz = get_M_grid(Ez, material_region, N_offset,1)
                Mx_vals[i,l,m,n,:,:,:] = Mx
                My_vals[i,l,m,n,:,:,:] = My
                Mz_vals[i,l,m,n,:,:,:] = Mz
            omega_vals[:,l,m,n] = omega
            vg_vals[:,l,m,n] = vgmag

In [ ]:
np.savez("phc_an_files/phc3d_sc_fs_e_12_30_12b.npz", BZ_sample, Mx_vals, My_vals, Mz_vals, omega_vals, vg_vals)

## BCC

In [ ]:
# body centered cubic lattice
sq3_2 = np.sqrt(3)/2
ms.geometry_lattice = mp.Lattice(basis_size=mp.Vector3(sq3_2,sq3_2,sq3_2),
                                basis1=mp.Vector3(1/2,1/2,-1/2),
                                basis2=mp.Vector3(-1/2,1/2,1/2),
                                basis3=mp.Vector3(1/2,-1/2,1/2))

# new 
Gamma = mp.Vector3(0,0,0)
H = mp.Vector3(-1/2,1/2,1/2)
N = mp.Vector3(0,1/2,0)
P = mp.Vector3(1/4,1/4,1/4)

# k_points = [N, H, Gamma, P, N, Gamma]
k_points = [P, Gamma, N, H]
ms.k_points = mp.interpolate(10, k_points)
ms.resolution = 32
num_bands = 12
ms.num_bands = 12
ms.run()
eps = np.array(ms.get_epsilon(), dtype=complex)
material_region = get_material_region(eps)
# imaterial_region = inv_material_region(eps)

# now everything is in the new vector basis
# and the M zone is in the basis of the new k basis


In [ ]:
plt.plot(ms.all_freqs, 'k')

In [ ]:
num_sample = 15
BZ_sample = np.linspace(-0.75,0.75, num_sample)
print(BZ_sample)

# number of points in mode integral to keep (which is in Fourier space, remember)
N_offset = 7

Mx_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
My_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
Mz_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
omega_vals = np.zeros([num_bands, num_sample, num_sample, num_sample])
vg_vals = np.zeros([num_bands, num_sample, num_sample, num_sample])
# converter = mpb.MPBData(rectify=True, periods=1, resolution=ms.resolution[0], lattice=ms.get_lattice())


for (l, kx) in enumerate(BZ_sample):
    for (m, ky) in enumerate(BZ_sample):
        for (n, kz) in enumerate(BZ_sample):
            if not (near_1BZ_BCC(BCC_to_cartesian_r @ [kx, ky, kz])):
                continue
            print(time.asctime(time.localtime()))
            print("(l, m, n) = ")
            print("(", str(l), str(m), str(n), ")")
            k = mp.Vector3(kx, ky, kz)
            E, omega, vgmag = get_mode_info(ms, k)
            Mbands = []
            for i in range(num_bands):
                Ex = np.array(E[i,0,:,:,:])
                Ey = np.array(E[i,1,:,:,:])
                Ez = np.array(E[i,2,:,:,:])
                Mx = get_M_grid(Ex, material_region, N_offset, 1/2)
                My = get_M_grid(Ey, material_region, N_offset, 1/2)
                Mz = get_M_grid(Ez, material_region, N_offset, 1/2)
                Mx_vals[i,l,m,n,:,:,:] = Mx
                My_vals[i,l,m,n,:,:,:] = My
                Mz_vals[i,l,m,n,:,:,:] = Mz
            omega_vals[:,l,m,n] = omega
            vg_vals[:,l,m,n] = vgmag

In [ ]:
# the data for 3d_bcc is saved in the BCC basis
np.savez("phc3d_bcc_fs_e_12_15_12b.npz", BZ_sample, Mx_vals, My_vals, Mz_vals, omega_vals, vg_vals)

## FCC free space

In [ ]:
sqr2 = np.sqrt(2)
ms.geometry_lattice = mp.Lattice(basis_size=mp.Vector3(1/sqr2, 1/sqr2,1/sqr2),
                                basis1=mp.Vector3(1,0,1),
                                basis2=mp.Vector3(1,1,0),
                                basis3=mp.Vector3(0,1,1))
ms.geometry = []

# in the reciprocal basis, not cartesian
k_points = [
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0.25, 0.75, 0.5),    # U
    mp.Vector3(0.5, 0.5, 0.5),      # L
    mp.Vector3(0, 0, 0),            # Gamma
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0.25, 0.75, 0.5),    # W
    mp.Vector3(0.375, 0.75, 0.375)  # K
]
ms.num_bands = 12
ms.k_points = mp.interpolate(10, k_points)
ms.run()
eps = np.array(ms.get_epsilon(), dtype=complex)
material_region = get_material_region(eps)


In [ ]:
np.savez("phc_an_files/phc3d_fcc_fs_bands_e_12_12.npz", ms.all_freqs)

## IOP

In [ ]:
geometry_lattice = mp.Lattice(
    basis_size=mp.Vector3(1/sqr2, 1/sqr2,1/sqr2),
                                basis1=mp.Vector3(1/2,0,1/2),
                                basis2=mp.Vector3(1/2,1/2,0),
                                basis3=mp.Vector3(0,1/2,1/2))
geometry = [mp.Sphere(0.43*np.sqrt(2), center=mp.Vector3(0,0,0), material=mp.Medium(epsilon=13.0)),
           mp.Sphere(0.36*np.sqrt(2), center=mp.Vector3(0,0,0), material=mp.Medium(epsilon=1))]
# geometry = [mp.Sphere(1/np.sqrt(8), center=mp.Vector3(0,0,0), material=mp.Medium(epsilon=1.0))]
num_bands = 12
resolution = 16       
k_points = [
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0.25, 0.75, 0.5),    # U
    mp.Vector3(0.5, 0.5, 0.5),      # L
    mp.Vector3(0, 0, 0),            # Gamma
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0.25, 0.75, 0.5),    # W
    mp.Vector3(0.375, 0.75, 0.375)  # K
]
k_points = mp.interpolate(10, k_points)
ms = mpb.ModeSolver(
    geometry=geometry,
    default_material=mp.Medium(epsilon=13.0),
    geometry_lattice=geometry_lattice,
    k_points=k_points,
    resolution=resolution,
    num_bands=num_bands
)

ms.run()
eps = np.array(ms.get_epsilon(), dtype=complex)
material_region = get_material_region(eps)


In [ ]:
plt.plot(ms.all_freqs)#* 2*np.pi*3e8/(0.5e-6))
plt.ylabel("Energy (eV)", fontsize=14)
plt.xlabel("")
plt.xticks([])
np.savez("phc_an_files/phc3d_bands_iop.npz", ms.all_freqs)

In [ ]:
num_sample = 15
BZ_sample = np.linspace(-0.75,0.75, num_sample)
print(BZ_sample)

# number of points in mode integral to keep (which is in Fourier space, remember)
N_offset = 7
vol_FCC = 1/4

Mx_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
My_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
Mz_vals = np.zeros([num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
omega_vals = np.zeros([num_bands, num_sample, num_sample, num_sample])
vg_vals = np.zeros([num_bands, num_sample, num_sample, num_sample])
# converter = mpb.MPBData(rectify=True, periods=1, resolution=ms.resolution[0], lattice=ms.get_lattice())


for (l, kx) in enumerate(BZ_sample):
    for (m, ky) in enumerate(BZ_sample):
        for (n, kz) in enumerate(BZ_sample):
            if not (near_1BZ_FCC(FCC_to_cartesian_r @ [kx, ky, kz])):
                continue
            print(time.asctime(time.localtime()))
            print("(l, m, n) = ")
            print("(", str(l), str(m), str(n), ")")
            k = mp.Vector3(kx, ky, kz)
            E, omega, vgmag = get_mode_info(ms, k)
            Mbands = []
            for i in range(num_bands):
                Ex = np.array(E[i,0,:,:,:])
                Ey = np.array(E[i,1,:,:,:])
                Ez = np.array(E[i,2,:,:,:])
                Mx = get_M_grid(Ex, material_region, N_offset, vol_FCC)
                My = get_M_grid(Ey, material_region, N_offset, vol_FCC)
                Mz = get_M_grid(Ez, material_region, N_offset, vol_FCC)
                Mx_vals[i,l,m,n,:,:,:] = Mx
                My_vals[i,l,m,n,:,:,:] = My
                Mz_vals[i,l,m,n,:,:,:] = Mz
            omega_vals[:,l,m,n] = omega
            vg_vals[:,l,m,n] = vgmag

In [ ]:
np.savez("phc3d_iop_15.npz", BZ_sample, Mx_vals, My_vals, Mz_vals, omega_vals, vg_vals)

## Woodpile

In [ ]:

ms.geometry_lattice = mp.Lattice(basis_size=mp.Vector3(np.sqrt(0.5), np.sqrt(0.5), np.sqrt(0.5)),
                             basis1=mp.Vector3(0,1,1),
                             basis2=mp.Vector3(1,0,1),
                             basis3=mp.Vector3(1,1,0))

def c(cart):
    return mp.cartesian_to_lattice(cart, geometry_lattice)
w = 0.2
h = 0.25
ms.resolution = 16
ms.default_material = mp.Medium(epsilon=1.0)
k_points = [
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0.25, 0.75, 0.5),    # U
    mp.Vector3(0.5, 0.5, 0.5),      # L
    mp.Vector3(0, 0, 0),            # Gamma
    mp.Vector3(0, 0.5, 0.5),        # X
    mp.Vector3(0.25, 0.75, 0.5),    # W
    mp.Vector3(0.375, 0.75, 0.375)  # K
]
ms.k_points = mp.interpolate(10, k_points)

ms.geometry = [mp.Block(center=c(mp.Vector3(0,0,0)), material=mp.Medium(epsilon=13.0), 
                     e1=c(mp.Vector3(1,1,0)),
                     e2=c(mp.Vector3(1,-1,0)),
                     e3=c(mp.Vector3(0,0,1)),
                     size=mp.Vector3(mp.inf, w, h)),
            mp.Block(center=c(mp.Vector3(0.125, 0.125, h)), material=mp.Medium(epsilon=13.0), 
                     e1=c(mp.Vector3(1,1,0)),
                     e2=c(mp.Vector3(1,-1,0)),
                     e3=c(mp.Vector3(0,0,1)),
                     size=mp.Vector3(w, mp.inf, h))
           ]
ms.run()

In [ ]:
plt.plot(ms.all_freqs) # * 2*np.pi*3e8/(0.5e-6) * 6.6e-16)
plt.ylabel("Energy", fontsize=14)
plt.xlabel("")
plt.xticks([])

In [ ]:
num_sample = 15
BZ_sample = np.linspace(-0.75,0.75, num_sample)
print(BZ_sample)

# number of points in mode integral to keep (which is in Fourier space, remember)
N_offset = 7
vol_FCC = 1/4

Mx_vals = np.zeros([ms.num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
My_vals = np.zeros([ms.num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
Mz_vals = np.zeros([ms.num_bands, num_sample, num_sample, num_sample, N_offset, N_offset, N_offset])
omega_vals = np.zeros([ms.num_bands, num_sample, num_sample, num_sample])
vg_vals = np.zeros([ms.num_bands, num_sample, num_sample, num_sample])
# converter = mpb.MPBData(rectify=True, periods=1, resolution=ms.resolution[0], lattice=ms.get_lattice())


for (l, kx) in enumerate(BZ_sample):
    for (m, ky) in enumerate(BZ_sample):
        for (n, kz) in enumerate(BZ_sample):
            if not (near_1BZ_FCC(FCC_to_cartesian_r @ [kx, ky, kz])):
                continue
            print(time.asctime(time.localtime()))
            print("(l, m, n) = ")
            print("(", str(l), str(m), str(n), ")")
            k = mp.Vector3(kx, ky, kz)
            E, omega, vgmag = get_mode_info(ms, k)
            Mbands = []
            for i in range(num_bands):
                Ex = np.array(E[i,0,:,:,:])
                Ey = np.array(E[i,1,:,:,:])
                Ez = np.array(E[i,2,:,:,:])
                Mx = get_M_grid(Ex, material_region, N_offset, vol_FCC)
                My = get_M_grid(Ey, material_region, N_offset, vol_FCC)
                Mz = get_M_grid(Ez, material_region, N_offset, vol_FCC)
                Mx_vals[i,l,m,n,:,:,:] = Mx
                My_vals[i,l,m,n,:,:,:] = My
                Mz_vals[i,l,m,n,:,:,:] = Mz
            omega_vals[:,l,m,n] = omega
            vg_vals[:,l,m,n] = vgmag

In [ ]:
np.savez("phc3d_wdpl_15.npz", BZ_sample, Mx_vals, My_vals, Mz_vals, omega_vals, vg_vals)